In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sqlalchemy import func
from tqdm import tqdm

print(os.getcwd())
os.chdir('../')
print(os.getcwd())
from src.buildDataset import build_static_dataset
from src.sqlstore.db import get_session
from src.sqlstore.match import SQLMatch
from src.sqlstore.timeline import SQLTimeline, SQLEvent, SQLKillEvent, SQLFrame, SQLParticipantFrame, \
    SQLTimelineDamageDealt, SQLTimelineDamageReceived
from src.utils import separateMatchID

/Users/admin/Documents/Informatik/Studium/Bachelorarbeit/LeagueOfLegendsWinPrediction/Exploration
/Users/admin/Documents/Informatik/Studium/Bachelorarbeit/LeagueOfLegendsWinPrediction


In [2]:
def build_frame_dataset(size: int, save: bool):
    with get_session() as session:
        matches = session.query(SQLMatch).order_by(func.random()).limit(size).all()
        matchIds = []
        frameData = []
        # iterate over matches
        for match in tqdm(matches):
            platformId, gameId = separateMatchID(match.matchId)

            # get timeline and frames for that match
            timeline = session.query(SQLTimeline).filter_by(platformId=platformId, gameId=gameId).first()
            frames = session.query(SQLFrame).filter_by(timelineId=timeline.id).all()
            participantFrameData = []
            # iterate over frames
            for frame in frames:
                matchIds.append((match.matchId, frame.id))
                frameDict = {'timestamp': frame.timestamp}
                participantFrames = session.query(SQLParticipantFrame).filter_by(frameId=frame.id).all()
                for i, participantFrame in enumerate(participantFrames):
                    participantFrameTrainingData = participantFrame.get_training_data()
                    participantFrameData.append(participantFrameTrainingData)
                    participantFrameDict = {f'participant{i + 1}_{k}': v for k, v in
                                            participantFrameTrainingData.items()}
                    frameDict.update(participantFrameDict)
                frameData.append(frameDict)
        index = pd.MultiIndex.from_tuples(matchIds, names=['matchId', 'frameId'])
        print(f"len of framedata: {len(frameData)}")
        print(f"shape of index: {index.shape}")
        print(f"len of matchIds: {len(matchIds)}")
        dfTimelines = pd.DataFrame(frameData, index=index)
        if save:
            dfTimelines.to_pickle('data/raw/timelines.pkl')
        return dfTimelines


In [3]:
data = build_frame_dataset(size=10, save=True)
data

100%|██████████| 10/10 [02:41<00:00, 16.11s/it]

len of framedata: 316
shape of index: (316,)
len of matchIds: 316


timestamp  participant1_participantId  \
matchId         frameId                                          
EUW1_6632739273 337920           0                           1   
                337921       60005                           1   
                337922      120012                           1   
                337923      180019                           1   
                337924      240024                           1   
...                            ...                         ...   
KR_6749764878   133942     1620489                           1   
                133943     1680506                           1   
                133944     1740520                           1   
                133945     1800524                           1   
                133946     1811779                           1   

                         participant1_abilityHaste  participant1_abilityPower  \
matchId         frameId                                                         
EUW1_6632739273 337920                           0                          0   
                337921                           0                          0   
                337922                           0                          0   
                337923                           0                          0   
                337924                           0                          0   
...                                            ...                        ...   
KR_6749764878   133942                           0                         30   
                133943                           0                         30   
                133944                           0                         30   
                133945                           0                         30   
                133946                           0                         30   

                         participant1_armor  participant1_armorPen  \
matchId         frameId                                              
EUW1_6632739273 337920                   26                      0   
                337921                   32                      0   
                337922                   32                      0   
                337923                   38                      0   
                337924                   41                      0   
...                                     ...                    ...   
KR_6749764878   133942                  144                      0   
                133943                  150                      0   
                133944                  150                      0   
                133945                  155                      0   
                133946                  155                      0   

                         participant1_armorPenPercent  \
matchId         frameId                                 
EUW1_6632739273 337920                              0   
                337921                              0   
                337922                              0   
                337923                              0   
                337924                              0   
...                                               ...   
KR_6749764878   133942                             10   
                133943                             10   
                133944                             10   
                133945                             15   
                133946                             15   

                         participant1_attackDamage  participant1_attackSpeed  \
matchId         frameId                                                        
EUW1_6632739273 337920                          25                       100   
                337921                          55                       113   
                337922                          55                       143   
                337923                          59         

In [58]:
def build_timeline_dataset(size: int, save: bool) -> pd.DataFrame:
    with get_session() as session:
        matches = session.query(SQLMatch).order_by(func.random()).limit(size).all()
        matchIds = []
        timelineIds = []
        for match in matches:
            matchIds.append(match.matchId)
            platformId, gameId = separateMatchID(match.matchId)
            timeline = session.query(SQLTimeline).filter_by(platformId=platformId, gameId=gameId).first()
            timelineIds.append(timeline.id)
            frames = session.query(SQLFrame).filter_by(timelineId=timeline.id).all()
            eventData = []
            frameIds = []
            eventIds = []
            for frame in frames:
                frameIds.append(frame.id)
                events = session.query(SQLEvent).filter_by(frameId=frame.id).all()
                # append all events in that frame to the eventData list
                for event in events:
                    eventIds.append(event.id)
                    eventData.append(event.get_training_data())
                killevents = session.query(SQLKillEvent).filter_by(frameId=frame.id).all()
                for killevent in killevents:
                    eventData.append(killevent.get_training_data())
                for participantFrame in frame.participantframe:
                    participantFrameData = participantFrame.get_training_data()
                    participantFrameData['timestamp'] = frame.timestamp
                    eventData.append(participantFrameData)
            eventData.sort(key=lambda x: x['timestamp'])
            print(eventData)
            df = pd.DataFrame(eventData)

            frameIds = []
            eventIds = []
            frame_timestamps = []
            event_Ids_timestamps = []
            participantFrames = []
            for frame in frames:
                frameIds.append(frame.id)
                frame_timestamps.append(frame.timestamp)
                for event in frame.events:
                    eventIds.append(event.id)
                    event_Ids_timestamps.append((event.id, event.timestamp))
                for participantFrame in frame.participantframe:
                    participantFrames.append(participantFrame.get_training_data())
            participants_list = [x for x in range(1, 11)]
            index_tuples = [(timeline.id, frame.id, participantId) for frame in frames for participantId in
                            participants_list]
            participantFrame_columns = participantFrames[0].keys()
            #print(participantFrame_columns)
            #print(index_tuples)
            index = pd.MultiIndex.from_tuples(index_tuples, names=['timelineId', 'frameId', 'participantId'])
            df = pd.DataFrame(participantFrames, index=index, columns=participantFrame_columns)
            return df
        #data = pd.DataFrame(zip(matchIds, timelineIds), columns=['matchId', 'timelineId'])

    return data

In [59]:
data = build_timeline_dataset(size=100, save=False)
data

[{'eventId': 0, 'timestamp': 0, 'type': 'PAUSE_END', 'participantId': None, 'itemId': None, 'skillSlot': None, 'creatorId': None, 'teamId': None, 'afterId': None, 'beforeId': None, 'wardType': None}, {'participantId': 1, 'abilityHaste': 0, 'abilityPower': 0, 'armor': 32, 'armorPen': 0, 'armorPenPercent': 0, 'attackDamage': 25, 'attackSpeed': 100, 'bonusArmorPenPercent': 0, 'bonusMagicPenPercent': 0, 'ccReduction': 0, 'cooldownReduction': 0, 'health': 625, 'healthMax': 625, 'healthRegen': 0, 'lifesteal': 0, 'magicPen': 0, 'magicPenPercent': 0, 'magicResist': 32, 'movementSpeed': 345, 'omnivamp': 0, 'physicalVamp': 0, 'power': 280, 'powerRegen': 0, 'spellVamp': 0, 'currentGold': 500, 'magicDamageDone': 0, 'magicDamageDoneToChampions': 0, 'magicDamageTaken': 0, 'physicalDamageDone': 0, 'physicalDamageDoneToChampions': 0, 'physicalDamageTaken': 0, 'totalDamageDone': 0, 'totalDamageDoneToChampions': 0, 'totalDamageTaken': 0, 'trueDamageDone': 0, 'trueDamageDoneToChampions': 0, 'trueDamageTa

participantId  abilityHaste  abilityPower  \
timelineId frameId participantId                                              
2013       57216   1                          1             0             0   
                   2                          2             0             0   
                   3                          3             0             0   
                   4                          4             0             0   
                   5                          5             0             0   
...                                         ...           ...           ...   
           57237   6                          6             0             0   
                   7                          7             0            44   
                   8                          8             0            68   
                   9                          9             0             0   
                   10                        10             0           248   

                                  armor  armorPen  armorPenPercent  \
timelineId frameId participantId                                     
2013       57216   1                 32         0                0   
                   2                 36         0                0   
                   3                 20         0                0   
                   4                 26         0                0   
                   5                 26         0                0   
...                                 ...       ...              ...   
           57237   6                115         0                0   
                   7                 77         0                0   
                   8                138         0                0   
                   9                 76         0                0   
                   10                65         0                0   

                                  attackDamage  attackSpeed  \
timelineId frameId participantId                              
2013       57216   1                        25          100   
                   2                        25          100   
                   3                        25          100   
                   4                        25          100   
                   5                        25          100   
...                                        ...          ...   
           57237   6                       214          183   
                   7                        76          129   
                   8                        98          127   
                   9                       232          175   
                   10                      101          140   

                                  bonusArmorPenPercent  bonusMagicPenPercent  \
timelineId frameId participantId                                               
2013       57216   1                                 0                     0   
                   2                                 0                     0   
                   3                                 0                     0   
                   4                                 0                     0   
                   5                                 0                     0   
...                                                ...                   ...   
           57237   6                                 0                     0   
                   7                                 0                     0   
                   8                                 0                     0   
                   9                                 0                     0   
                   10                                0                     0   

                                  ...  trueDamageTaken  goldPerSecond  \
timelineId frameId participantId  ...                                   
2013       57216   1              ...                0              0   
               